In [1]:
import numpy as np
import pandas as pd
from math import sqrt
from scipy.stats import norm
from matplotlib import pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
from tqdm import tqdm
import scipy as sc

In [2]:
# !pip install plotly --upgrade

In [3]:
def get_x_y():
    x= [x for x in range(1,501)]
    y = [sqrt(0.05*k)+np.random.normal(0,1,1) for k in range(1,501)]
    return x,y
# x= [x for x in range(1,501)]
# y = [sqrt(0.05*k)+np.random.normal(0,1,1) for k in x]

In [4]:
x_, y_ = get_x_y()
y_ = np.concatenate(y_, axis=0)
# sqrt_y = np.sqrt(y_)
sqrt_x = [sqrt(0.05*k) for k in x_]
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_, y=y_, mode='markers', name='$$Dots$$'))
fig.add_trace(go.Scatter(x=x_, y=sqrt_x, name='$f(x) = \sqrt{x}$'))
fig.update_layout(
                  title="First visualization",
                  xaxis_title="X",
                  yaxis_title="Y",)
#                   margin=dict(l=0, r=0, t=30, b=0))
fig.show()

In [5]:
def moving_averanges(m,x,y):
    for i in range(len(y)-m+1):
        yield 1/(m)*sum(y[i:i+m])

def moving_medians(m,x,y):
    for i in range(len(y)-m+1):
        yield np.median(y[i:i+m])

In [6]:
def moving_med(sample, window: int):
    gap = int((window - 1) / 2)
    output = [np.median([sample[0], sample[1], 3*sample[1] - 2*sample[2]])]
    for i in range(1, gap):
        output.append(np.median(sample[:2*i]))
    for i in range(gap, len(sample) - gap):
        output.append(np.median(sample[i-gap:i+gap]))
    for i in range(len(sample) - gap, len(sample)):
        output.append(np.median(sample[2*i - 2*len(sample):]))  # negative index

    output[-1] = np.median([sample[-1], sample[-2], 3*sample[-2] - 2*sample[-3]])
    return output

In [7]:
def moving_avg(sample, window: int):
    gap = int((window - 1) / 2)
    output = [sample[0]]
    for i in range(1, gap):
        output.append(np.mean(sample[:2*i]))
    for i in range(gap, len(sample) - gap):
        output.append(np.mean(sample[i-gap:i+gap]))
    for i in range(len(sample) - gap, len(sample)):
        output.append(np.mean(sample[2*i - 2*len(sample):]))  # negative index

    return output

In [8]:
def get_slice_index(m):
    return int((m-1)/2)

In [9]:
def get_graph_averanges(arr_of_m):
    fig = go.Figure()
    for m in arr_of_m:
        x, y = get_x_y()
#         x = list(moving_medians(500, _x, _y))
        border = get_slice_index(m)
        fig.add_trace(go.Scatter(x=x[1:], y=moving_avg(y, m)[1:],name=f'Ширина окна = {m}'))
    sqrt_x = [sqrt(0.05*k) for k in x_]
    fig.add_trace(go.Scatter(x=x_, y=y_, mode='markers', name='$$Dots$$'))
    fig.add_trace(go.Scatter(x=x_, y=sqrt_x, name='$f(x) = \sqrt{x}$'))
    fig.update_layout(
                      title="Moving averanges",
                      xaxis_title="X",
                      yaxis_title="Y",)
    fig.show()

In [10]:
get_graph_averanges([11,21,51,111])

In [11]:
def get_graph_medians(arr_of_m):
    x = moving_medians
    fig = go.Figure()
    x,y = get_x_y()
    for m in arr_of_m:
        border = get_slice_index(m)
        fig.add_trace(go.Scatter(x=x[1:], y=moving_med(y,m)[1:],name=f'Ширина окна = {m}'))
    sqrt_x = [sqrt(0.05*k) for k in x_]
    fig.add_trace(go.Scatter(x=x_, y=y_, mode='markers', name='$$Dots$$'))
    fig.add_trace(go.Scatter(x=x_, y=sqrt_x, name='$f(x) = \sqrt{x}$'))
    fig.update_layout(
                      title="Moving medians",
                      xaxis_title="X",
                      yaxis_title="Y",)
    fig.show()

In [12]:
get_graph_medians([11,21,51,111])

In [14]:
def get_graph_compare(m):
    fig = go.Figure()
    x, y = get_x_y()
    fig.add_trace(go.Scatter(x=x_, y=y_, mode='markers', name='$$Dots$$'))
    fig.add_trace(go.Scatter(x=x_, y=sqrt_x, name='$f(x) = \sqrt{x}$'))
    border = get_slice_index(m)
    fig.add_trace(go.Scatter(x=x[1:], y=moving_med(y,m)[1:],name=f'Ширина окна = {m}, medians'))
    fig.add_trace(go.Scatter(x=x[1:], y=moving_avg(y, m)[1:],name=f'Ширина окна = {m}, averanges'))
    fig.update_layout(
                      title="Compare medians and averanges",
                      xaxis_title="X",
                      yaxis_title="Y",)
    fig.show()

In [15]:
get_graph_compare(111)

In [16]:
def get_noise(n, x, y, trend):
    border = get_slice_index(n)
    y = np.array(y)
#     trend = np.array(list(moving_averanges(n,x,y)))
    noise = y[1:] - trend[1:]
    return noise


In [17]:
def countof_turningpoints(lst):
    dx = np.diff(lst,axis=0)
    return np.sum(dx[1:] * dx[:-1] < 0)


In [ ]:
# trend = moving_avg(y,11)
# a = get_noise(11, x, y, trend)
# countof_turningpoints(a)

In [18]:
def criterion_of_turning_point(n):
    x,y = get_x_y()
    y = np.concatenate(y, axis=0)
    trend = moving_avg(y,n)
#     trend = moving_med(y,n)
#     trend = moving_avg_BOOST(y,n)
    noise = get_noise(n, x, y, trend)
    points_count = countof_turningpoints(noise)
    border = get_slice_index(n)
    treshold = 2/3*len(x[1:])
    return points_count, treshold

In [29]:
turning_points, treshold = criterion_of_turning_point(111)
dispersion_turning_points = sqrt((16*500 - 29)/500)
fig = go.Figure(data=[go.Bar(x = ['Количество поворотных точек', '2/3'], y = [turning_points, treshold])])
fig.add_shape(type='line',
                x0=-1,
                y0=dispersion_turning_points+treshold,
                x1=2,
                y1=dispersion_turning_points+treshold,
                line=dict(color='Red',),
                xref='x',
                yref='y'
)
fig.add_shape(type='line',
                x0=-1,
                y0=-dispersion_turning_points+treshold,
                x1=2,
                y1=-dispersion_turning_points+treshold,
                line=dict(color='Red',),
                xref='x',
                yref='y'
             )
fig.update_layout(
                      title=f"Поворотных точек - {turning_points}, 2/3 - {treshold}, std - {dispersion_turning_points}",
                      xaxis_title="X",
                      yaxis_title="Y",)

fig.show()

In [21]:
def init_inv_N_matrix(polynom_extent, window: int):
    abscissa_values = range(-int((window - 1)/2), int((window - 1)/2) + 1)
    N = np.empty([polynom_extent + 1, polynom_extent + 1])
    for i in range(N.shape[0]):
        for j in range(i, N.shape[1]):
            N[i][j] = sum([item ** (i+j) for item in abscissa_values])
            N[j][i] = sum([item ** (i+j) for item in abscissa_values])

    return sc.linalg.inv(N)

def init_X_vector(x_slice, polynom_extent):
    abscissa_values = range(-int((len(x_slice) - 1)/2), int((len(x_slice) - 1)/2) + 1)
    X = np.array([0]*(polynom_extent + 1))
    for i in range(polynom_extent + 1):
        X[i] = sum([abscissa_values[k] ** i * x_slice[k] for k in range(len(abscissa_values))])  # ToDo range

    return X

def moving_avg_BOOST(sample, window, polynom_extent = 1):
    N_inv = init_inv_N_matrix(polynom_extent, window)

    gap = int((window - 1) / 2)
    output = [sample[0]]
    for i in range(1, gap):
        output.append(np.mean(sample[:2*i]))
    for i in range(gap, len(sample) - gap):
        X = init_X_vector(sample[i-gap:i+gap+1], polynom_extent)
        A = N_inv.dot(X)
        output.append(A[0])
    for i in range(len(sample) - gap, len(sample)):
        output.append(np.mean(sample[2*i - 2*len(sample):]))  # negative index

    return output

In [22]:
def count_P(arr):
    count = 0
    for i in range(len(arr)):
        for j in range(i):
            if arr[i] > arr[j]:
                count +=1
    return count

def get_Kendal(n,m, trend):
    x,y= get_x_y()
    y = np.concatenate(y, axis=0)
#     trend = moving_med(y,m)
#     trend = moving_avg(y,m)
#     trend = moving_avg_BOOST(21, 3)
    noise = get_noise(n, x, y,trend)
    P = count_P(noise)
    return 4*P/(n*(n-1)) -1

def get_dispersion_Kendal(n):
    return sqrt(2*(2*n+5)/(9*n*(n-1)))

In [23]:
x, y = get_x_y()
m = 111
trend_med = moving_med(y,m)
trend_avg = moving_avg(y,m)
# trend_pilinoms = moving_avg_BOOST(111, 3)
# n = 500
# m = 111
print(f'Коэффициент Кендала для медианы = {get_Kendal(500,m,trend_med )}')
print(f'Коэффициент Кендала для среднего = {get_Kendal(500,m,trend_avg )}')
# print(f'Коэффициент Кендала для пилинома = {get_Kendal(500,m,trend_pilinoms )}')
print(f'Дисперсия = {get_dispersion_Kendal(500)}')

Коэффициент Кендала для медианы = 0.050757515030060096
Коэффициент Кендала для среднего = 0.04498597194388787
Дисперсия = 0.02991861595218472


In [24]:
# def create_samples(trend, my_range = 20):

#     dict_ = {11: [], 21: [], 51: [], 111: []}
#     for coef in tqdm([11, 21, 51, 111]):

#         for j in range(my_range):
#             x, y = get_x_y()
#             trend_ = trend(y, coef)
#             dict_[coef].append(get_Kendal(500, coef,trend_))
#     return dict_


In [25]:
# trends = [moving_avg,moving_med,moving_avg_BOOST]
# arr_dicts = [create_samples(trend) for trend in trends]
# # dict_ = create_samples()
# # fig = go.Figure()
# fig = make_subplots(rows=3, cols=1, subplot_titles=('Moving_medians', 'Moving_averanges', 'Ours_Pilinoms'))
# for num, dict_ in enumerate(arr_dicts):
#     print(num)
#     my_range = 20
#     for key, value in dict_.items():
#         fig.add_trace(go.Scatter(x=list(range(my_range)), y=value, name=f'Размер окна = {key}', legendgroup=num+1), row=num+1, col=1)
#     fig.add_shape(type='line',
#                     x0=0,
#                     y0=-get_dispersion_Kendal(500),
#                     x1=my_range,
#                     y1=-get_dispersion_Kendal(500),
#                     line=dict(color='Black',),
#                     xref='x',
#                     yref='y',row=num+1, col=1
#                  )
#     fig.add_shape(type='line',
#                     x0=0,
#                     y0=get_dispersion_Kendal(500),
#                     x1=my_range,
#                     y1=get_dispersion_Kendal(500),
#                     line=dict(color='Black',),
#                     xref='x',
#                     yref='y', row=num+1, col=1
#                  )
# fig.update_layout(height=1200, width=1000, title_text="Stacked Subplots", legend_tracegroupgap = 360)

# fig.show()

100%|██████████| 4/4 [00:09<00:00,  2.39s/it]

0
1
2
